<div>
<img src=https://www.institutedata.com/wp-content/uploads/2019/10/iod_h_tp_primary_c.svg width="300">
</div>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Lab 9.7: Text Classification
INSTRUCTIONS:
- Run the cells
- Observe and understand the results
- Answer the questions

## Import libraries

In [4]:
## Import Libraries
import numpy as np
import pandas as pd

import string
import spacy

from collections import Counter

from sklearn.decomposition import LatentDirichletAllocation
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC

# import warnings
# warnings.filterwarnings('ignore')

## Load data

Sample:

    __label__2 Stuning even for the non-gamer: This sound ...
    __label__2 The best soundtrack ever to anything.: I'm ...
    __label__2 Amazing!: This soundtrack is my favorite m ...
    __label__2 Excellent Soundtrack: I truly like this so ...
    __label__2 Remember, Pull Your Jaw Off The Floor Afte ...
    __label__2 an absolute masterpiece: I am quite sure a ...
    __label__1 Buyer beware: This is a self-published boo ...
    . . .
    
There are only two **labels**:
- `__label__1`
- `__label__2`

In [6]:
## Loading the data

trainDF = pd.read_fwf(
    filepath_or_buffer = '/content/corpus.txt',
    colspecs = [(9, 10),   # label: get only the numbers 1 or 2
                (11, 9000) # text: makes the it big enought to get to the end of the line
               ], 
    header = 0,
    names = ['label', 'text'],
    lineterminator = '\n'
)

# convert label from [1, 2] to [0, 1]
trainDF['label'] = trainDF['label'] - 1

## Inspect the data

In [7]:
# ANSWER
trainDF

,label,text
0,1,The best soundtrack ever to anything.: I'm rea...
1,1,Amazing!: This soundtrack is my favorite music...
2,1,Excellent Soundtrack: I truly like this soundt...
3,1,"Remember, Pull Your Jaw Off The Floor After He..."
4,1,an absolute masterpiece: I am quite sure any o...
...,...,...
9994,1,A revelation of life in small town America in ...
9995,1,Great biography of a very interesting journali...
9996,0,Interesting Subject; Poor Presentation: You'd ...
9997,0,Don't buy: The box looked used and it is obvio...


## Split the data into train and test

In [8]:
## ANSWER
## split the dataset
X_train, X_test, y_train, y_test = train_test_split(trainDF.text, trainDF.label, test_size = 0.2, random_state= 42)

## Feature Engineering

### Count Vectors as features

In [9]:
# create a count vectorizer object
count_vect = CountVectorizer(token_pattern = r'\w{1,}')

# Learn a vocabulary dictionary of all tokens in the raw documents
count_vect.fit(trainDF['text'])

# Transform documents to document-term matrix.
X_train_count = count_vect.transform(X_train)
X_test_count = count_vect.transform(X_test)

### TF-IDF Vectors as features
- Word level
- N-Gram level
- Character level

In [10]:
%%time
# word level tf-idf
tfidf_vect = TfidfVectorizer(analyzer = 'word',
                             token_pattern = r'\w{1,}',
                             max_features = 5000)
print(tfidf_vect)

tfidf_vect.fit(trainDF['text'])
X_train_tfidf = tfidf_vect.transform(X_train)
X_test_tfidf  = tfidf_vect.transform(X_test)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=5000,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='\\w{1,}', tokenizer=None,
                use_idf=True, vocabulary=None)
CPU times: user 1.23 s, sys: 7.93 ms, total: 1.24 s
Wall time: 1.25 s


In [11]:
%%time
# ngram level tf-idf
tfidf_vect_ngram = TfidfVectorizer(analyzer = 'word',
                                   token_pattern = r'\w{1,}',
                                   ngram_range = (2, 3),
                                   max_features = 5000)
print(tfidf_vect_ngram)

tfidf_vect_ngram.fit(trainDF['text'])
X_train_tfidf_ngram = tfidf_vect_ngram.transform(X_train)
X_test_tfidf_ngram  = tfidf_vect_ngram.transform(X_test)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=5000,
                min_df=1, ngram_range=(2, 3), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='\\w{1,}', tokenizer=None,
                use_idf=True, vocabulary=None)
CPU times: user 6.46 s, sys: 212 ms, total: 6.68 s
Wall time: 6.69 s


In [12]:
%%time
# characters level tf-idf
tfidf_vect_ngram_chars = TfidfVectorizer(analyzer = 'char',
                                         token_pattern = r'\w{1,}',
                                         ngram_range = (2, 3),
                                         max_features = 5000)
print(tfidf_vect_ngram_chars)

tfidf_vect_ngram_chars.fit(trainDF['text'])
X_train_tfidf_ngram_chars = tfidf_vect_ngram_chars.transform(X_train)
X_test_tfidf_ngram_chars  = tfidf_vect_ngram_chars.transform(X_test)

TfidfVectorizer(analyzer='char', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=5000,
                min_df=1, ngram_range=(2, 3), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='\\w{1,}', tokenizer=None,
                use_idf=True, vocabulary=None)


/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:524: UserWarning: The parameter 'token_pattern' will not be used since 'analyzer' != 'word'
  warnings.warn("The parameter 'token_pattern' will not be used"


CPU times: user 9.65 s, sys: 36.5 ms, total: 9.69 s
Wall time: 9.71 s


### Text / NLP based features

Create some other features.

Char_Count = Number of Characters in Text

Word Count = Number of Words in Text

Word Density = Average Number of Char in Words

Punctuation Count = Number of Punctuation in Text

Title Word Count = Number of Words in Title

Uppercase Word Count = Number of Upperwords in Text

In [13]:
import string
def char_count(text):
  return len(''.join(text.split(' ')))

def word_count(text):
  return len(text.split(' '))

def word_density(text):
  return char_count(text) / word_count(text)

def punct_count(text):
  count = lambda l1, l2: len(list(filter(lambda c: c in l2, l1)))
  return count(text, string.punctuation)

def upper_word_count(text):
  count = 0
  for t in text.split(' '):
    if t[0].isupper():
      count += 1
  return count

def title_count(text):
  count = 0
  words = text.split()
  for word in words:
    if word == words[0] and words[0].istitle() and words[1].istitle() and words[1] != 'I':
      count += 1
    elif word.istitle() and word != 'I' and word != words[0]:
      count += 1
  return count

In [14]:
functions = [char_count, word_count, word_density, punct_count, upper_word_count, title_count]
function_names = ['char_count', 'word_count', 'word_density', 'punct_count', 'upper_word_count', 'title_count']

for i in range(len(functions)):
  trainDF[function_names[i]] = trainDF['text'].apply(functions[i])


In [15]:
trainDF

,label,text,char_count,word_count,word_density,punct_count,upper_word_count,title_count
0,1,The best soundtrack ever to anything.: I'm rea...,413,97,4.257732,14,11,2
1,1,Amazing!: This soundtrack is my favorite music...,632,129,4.899225,40,18,22
2,1,Excellent Soundtrack: I truly like this soundt...,626,118,5.305085,33,54,48
3,1,"Remember, Pull Your Jaw Off The Floor After He...",395,87,4.540230,22,30,30
4,1,an absolute masterpiece: I am quite sure any o...,684,142,4.816901,35,13,12
...,...,...,...,...,...,...,...,...
9994,1,A revelation of life in small town America in ...,716,152,4.710526,25,16,12
9995,1,Great biography of a very interesting journali...,721,141,5.113475,14,16,15
9996,0,Interesting Subject; Poor Presentation: You'd ...,543,108,5.027778,17,12,11
9997,0,Don't buy: The box looked used and it is obvio...,109,27,4.037037,6,4,1


In [16]:
## load spaCy
nlp = spacy.load('en')

Part of Speech in **SpaCy**

    POS   DESCRIPTION               EXAMPLES
    ----- ------------------------- ---------------------------------------------
    ADJ   adjective                 big, old, green, incomprehensible, first
    ADP   adposition                in, to, during
    ADV   adverb                    very, tomorrow, down, where, there
    AUX   auxiliary                 is, has (done), will (do), should (do)
    CONJ  conjunction               and, or, but
    CCONJ coordinating conjunction  and, or, but
    DET   determiner                a, an, the
    INTJ  interjection              psst, ouch, bravo, hello
    NOUN  noun                      girl, cat, tree, air, beauty
    NUM   numeral                   1, 2017, one, seventy-seven, IV, MMXIV
    PART  particle                  's, not,
    PRON  pronoun                   I, you, he, she, myself, themselves, somebody
    PROPN proper noun               Mary, John, London, NATO, HBO
    PUNCT punctuation               ., (, ), ?
    SCONJ subordinating conjunction if, while, that
    SYM   symbol                    $, %, §, ©, +, −, ×, ÷, =, :), 😝
    VERB  verb                      run, runs, running, eat, ate, eating
    X     other                     sfpksdpsxmsa
    SPACE space
    
Find out number of Adjective, Adverb, Noun, Numeric, Pronoun, Proposition, Verb.

    Hint:
    1. Convert text to spacy document
    2. Use pos_
    3. Use Counter 

In [17]:
# Initialise some columns for feature's counts
trainDF['adj_count'] = 0
trainDF['adv_count'] = 0
trainDF['noun_count'] = 0
trainDF['num_count'] = 0
trainDF['pron_count'] = 0
trainDF['propn_count'] = 0
trainDF['verb_count'] = 0
trainDF

,label,text,char_count,word_count,word_density,punct_count,upper_word_count,title_count,adj_count,adv_count,noun_count,num_count,pron_count,propn_count,verb_count
0,1,The best soundtrack ever to anything.: I'm rea...,413,97,4.257732,14,11,2,0,0,0,0,0,0,0
1,1,Amazing!: This soundtrack is my favorite music...,632,129,4.899225,40,18,22,0,0,0,0,0,0,0
2,1,Excellent Soundtrack: I truly like this soundt...,626,118,5.305085,33,54,48,0,0,0,0,0,0,0
3,1,"Remember, Pull Your Jaw Off The Floor After He...",395,87,4.540230,22,30,30,0,0,0,0,0,0,0
4,1,an absolute masterpiece: I am quite sure any o...,684,142,4.816901,35,13,12,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9994,1,A revelation of life in small town America in ...,716,152,4.710526,25,16,12,0,0,0,0,0,0,0
9995,1,Great biography of a very interesting journali...,721,141,5.113475,14,16,15,0,0,0,0,0,0,0
9996,0,Interesting Subject; Poor Presentation: You'd ...,543,108,5.027778,17,12,11,0,0,0,0,0,0,0
9997,0,Don't buy: The box looked used and it is obvio...,109,27,4.037037,6,4,1,0,0,0,0,0,0,0


In [18]:
trainDF

,label,text,char_count,word_count,word_density,punct_count,upper_word_count,title_count,adj_count,adv_count,noun_count,num_count,pron_count,propn_count,verb_count
0,1,The best soundtrack ever to anything.: I'm rea...,413,97,4.257732,14,11,2,0,0,0,0,0,0,0
1,1,Amazing!: This soundtrack is my favorite music...,632,129,4.899225,40,18,22,0,0,0,0,0,0,0
2,1,Excellent Soundtrack: I truly like this soundt...,626,118,5.305085,33,54,48,0,0,0,0,0,0,0
3,1,"Remember, Pull Your Jaw Off The Floor After He...",395,87,4.540230,22,30,30,0,0,0,0,0,0,0
4,1,an absolute masterpiece: I am quite sure any o...,684,142,4.816901,35,13,12,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9994,1,A revelation of life in small town America in ...,716,152,4.710526,25,16,12,0,0,0,0,0,0,0
9995,1,Great biography of a very interesting journali...,721,141,5.113475,14,16,15,0,0,0,0,0,0,0
9996,0,Interesting Subject; Poor Presentation: You'd ...,543,108,5.027778,17,12,11,0,0,0,0,0,0,0
9997,0,Don't buy: The box looked used and it is obvio...,109,27,4.037037,6,4,1,0,0,0,0,0,0,0


In [19]:
# ANSWER
row_nums = np.arange(8,15)
features = ['ADJ','ADV','NOUN','NUM','PRON','PROPN','VERB']

for index, cols in trainDF.iterrows():
  doc = nlp(trainDF.loc[index,'text'])
  counter = Counter({'ADJ':0,'ADV':0,'NOUN':0,'NUM':0,'PRON':0,'PROPN':0,'VERB':0})
  for token in doc:
    if token.pos_ in counter.keys():
      counter[token.pos_] += 1
  for i in range(len(counter.keys())):
    trainDF.loc[index, trainDF.columns[row_nums[i]]] = counter[features[i]]
trainDF



,label,text,char_count,word_count,word_density,punct_count,upper_word_count,title_count,adj_count,adv_count,noun_count,num_count,pron_count,propn_count,verb_count
0,1,The best soundtrack ever to anything.: I'm rea...,413,97,4.257732,14,11,2,7,4,18,1,9,2,15
1,1,Amazing!: This soundtrack is my favorite music...,632,129,4.899225,40,18,22,13,9,22,1,11,13,13
2,1,Excellent Soundtrack: I truly like this soundt...,626,118,5.305085,33,54,48,8,4,16,4,5,37,8
3,1,"Remember, Pull Your Jaw Off The Floor After He...",395,87,4.540230,22,30,30,6,3,12,0,8,16,11
4,1,an absolute masterpiece: I am quite sure any o...,684,142,4.816901,35,13,12,17,14,20,2,10,10,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9994,1,A revelation of life in small town America in ...,716,152,4.710526,25,16,12,16,7,36,0,4,10,17
9995,1,Great biography of a very interesting journali...,721,141,5.113475,14,16,15,15,5,38,0,3,11,10
9996,0,Interesting Subject; Poor Presentation: You'd ...,543,108,5.027778,17,12,11,16,3,25,0,7,2,14
9997,0,Don't buy: The box looked used and it is obvio...,109,27,4.037037,6,4,1,2,1,3,0,4,0,5


### Topic Models as features

In [20]:
%%time
# train a LDA Model
lda_model = LatentDirichletAllocation(n_components = 20, learning_method = 'online', max_iter = 20)

X_topics = lda_model.fit_transform(X_train_count)
topic_word = lda_model.components_ 
vocab = count_vect.get_feature_names()

CPU times: user 1min 30s, sys: 206 ms, total: 1min 30s
Wall time: 1min 30s


In [21]:
vocab

['0',
 '00',
 '000',
 '001',
 '002',
 '00290',
 '007',
 '0070412901',
 '0072316373',
 '008',
 '00now',
 '00yeah',
 '01',
 '011',
 '02',
 '03',
 '04',
 '05',
 '052',
 '06',
 '06rate',
 '07',
 '088',
 '09',
 '0s9',
 '0sx',
 '1',
 '10',
 '100',
 '1000',
 '1000amp',
 '1000s',
 '1000uf',
 '1001',
 '100m',
 '100th',
 '100this',
 '101',
 '10162',
 '102',
 '1020',
 '1021',
 '103',
 '1030pm',
 '104',
 '1048259',
 '105',
 '1058',
 '1059',
 '107',
 '108',
 '1080p',
 '10bm',
 '10ft',
 '10gameplay',
 '10gb',
 '10i',
 '10mb',
 '10min',
 '10movie',
 '10overall',
 '10p',
 '10sound',
 '10th',
 '10the',
 '10this',
 '11',
 '110',
 '1100',
 '110e',
 '1115',
 '11223',
 '114622',
 '115',
 '116',
 '119',
 '11b',
 '11g',
 '11i',
 '11lbs',
 '11movie',
 '11th',
 '12',
 '120',
 '1200',
 '120100',
 '120lbs',
 '120s',
 '120v',
 '121',
 '1215',
 '123',
 '125',
 '128',
 '128mb',
 '129',
 '12th',
 '12x',
 '13',
 '130',
 '133x',
 '1340s',
 '135lbs',
 '1394',
 '13th',
 '13w',
 '13x',
 '14',
 '140',
 '140lbs',
 '144',
 

In [22]:
# view the topic models
n_top_words = 10
topic_summaries = []
print('Group Top Words')
print('-----', '-'*80)
for i, topic_dist in enumerate(topic_word):
    topic_words = np.array(vocab)[np.argsort(topic_dist)][:-(n_top_words+1):-1]
    top_words = ' '.join(topic_words)
    topic_summaries.append(top_words)
    print('  %3d %s' % (i, top_words))

Group Top Words
----- --------------------------------------------------------------------------------
    0 mad spanish dubbed violence italy mel acts folk gibson disease
    1 camcorder album techno latest occasionally fool seek advise standards et
    2 dialogue bottle planet eating warmer marks index episode paris captured
    3 installation funk huppert philadelphia mike torn buddhism beats kentucky ambient
    4 japanese sleeping theory office catholic zen peace japan heater humanity
    5 la de y en el que con harry los un
    6 cap aluminum ra oriented frm gillain garp jets crafty lego
    7 trash receive rod shallow project worthy wanting allowed shakespeare keel
    8 anderson steer shades eyed stalingrad inducing greek pulp outs joanna
    9 season show episodes shirt ballet scratched csi kills 28 yer
   10 power printer adapter apple laptop g4 computer ibook macally mac
   11 camera card battery charger works with work canon sony cable
   12 grammar everest explaining dali 

## Modelling

In [23]:
## helper function

def train_model(classifier, feature_vector_train, label, feature_vector_valid):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)

    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)

    return accuracy_score(predictions, y_test)

In [24]:
# Keep the results in a dataframe
results = pd.DataFrame(columns = ['Count Vectors',
                                  'WordLevel TF-IDF',
                                  'N-Gram Vectors',
                                  'CharLevel Vectors'])

### Naive Bayes Classifier

In [25]:
%%time
# Naive Bayes on Count Vectors
accuracy1 = train_model(MultinomialNB(), X_train_count, y_train, X_test_count)
print('NB, Count Vectors    : %.4f\n' % accuracy1)

NB, Count Vectors    : 0.8540

CPU times: user 10 ms, sys: 0 ns, total: 10 ms
Wall time: 13.2 ms


In [26]:
%%time
# Naive Bayes on Word Level TF IDF Vectors
accuracy2 = train_model(MultinomialNB(), X_train_tfidf, y_train, X_test_tfidf)
print('NB, WordLevel TF-IDF : %.4f\n' % accuracy2)

NB, WordLevel TF-IDF : 0.8600

CPU times: user 10.3 ms, sys: 0 ns, total: 10.3 ms
Wall time: 14.1 ms


In [27]:
%%time
# Naive Bayes on Ngram Level TF IDF Vectors
accuracy3 = train_model(MultinomialNB(), X_train_tfidf_ngram, y_train, X_test_tfidf_ngram)
print('NB, N-Gram Vectors   : %.4f\n' % accuracy3)

NB, N-Gram Vectors   : 0.8400

CPU times: user 8.06 ms, sys: 0 ns, total: 8.06 ms
Wall time: 13.6 ms


In [28]:
%%time
# # Naive Bayes on Character Level TF IDF Vectors
accuracy4 = train_model(MultinomialNB(), X_train_tfidf_ngram_chars, y_train, X_test_tfidf_ngram_chars)
print('NB, CharLevel Vectors: %.4f\n' % accuracy4)

NB, CharLevel Vectors: 0.8180

CPU times: user 32.8 ms, sys: 0 ns, total: 32.8 ms
Wall time: 32.9 ms


In [29]:
results.loc['Naïve Bayes'] = {
    'Count Vectors': accuracy1,
    'WordLevel TF-IDF': accuracy2,
    'N-Gram Vectors': accuracy3,
    'CharLevel Vectors': accuracy4}

### Linear Classifier

In [30]:
%%time
# Linear Classifier on Count Vectors
accuracy1 = train_model(LogisticRegression(solver = 'lbfgs', max_iter = 350), X_train_count, y_train, X_test_count)
print('LR, Count Vectors    : %.4f\n' % accuracy1)

LR, Count Vectors    : 0.8520

CPU times: user 2.95 s, sys: 3.4 s, total: 6.35 s
Wall time: 3.26 s


In [31]:
%%time
# Linear Classifier on Word Level TF IDF Vectors
accuracy2 = train_model(LogisticRegression(solver = 'lbfgs', max_iter = 100), X_train_tfidf, y_train, X_test_tfidf)
print('LR, WordLevel TF-IDF : %.4f\n' % accuracy2)

LR, WordLevel TF-IDF : 0.8730

CPU times: user 135 ms, sys: 127 ms, total: 262 ms
Wall time: 156 ms


In [32]:
%%time
# Linear Classifier on Ngram Level TF IDF Vectors
accuracy3 = train_model(LogisticRegression(solver = 'lbfgs', max_iter = 100), X_train_tfidf_ngram, y_train, X_test_tfidf_ngram)
print('LR, N-Gram Vectors   : %.4f\n' % accuracy3)

LR, N-Gram Vectors   : 0.8360

CPU times: user 55.2 ms, sys: 2 µs, total: 55.2 ms
Wall time: 55.2 ms


In [33]:
%%time
# Linear Classifier on Character Level TF IDF Vectors
accuracy4 = train_model(LogisticRegression(solver = 'lbfgs', max_iter = 100), X_train_tfidf_ngram_chars, y_train, X_test_tfidf_ngram_chars)
print('LR, CharLevel Vectors: %.4f\n' % accuracy4)

LR, CharLevel Vectors: 0.8485

CPU times: user 420 ms, sys: 13 µs, total: 420 ms
Wall time: 420 ms


In [34]:
results.loc['Logistic Regression'] = {
    'Count Vectors': accuracy1,
    'WordLevel TF-IDF': accuracy2,
    'N-Gram Vectors': accuracy3,
    'CharLevel Vectors': accuracy4}

### Support Vector Machine

In [35]:
%%time
# Support Vector Machine on Count Vectors
accuracy1 = train_model(LinearSVC(), X_train_count, y_train, X_test_count)
print('SVM, Count Vectors    : %.4f\n' % accuracy1)

SVM, Count Vectors    : 0.8345

CPU times: user 536 ms, sys: 0 ns, total: 536 ms
Wall time: 546 ms


In [36]:
%%time
# Support Vector Machine on Word Level TF IDF Vectors
accuracy2 = train_model(LinearSVC(), X_train_tfidf, y_train, X_test_tfidf)
print('SVM, WordLevel TF-IDF : %.4f\n' % accuracy2)

SVM, WordLevel TF-IDF : 0.8610

CPU times: user 67.7 ms, sys: 995 µs, total: 68.7 ms
Wall time: 71.3 ms


In [37]:
%%time
# Support Vector Machine on Ngram Level TF IDF Vectors
accuracy3 = train_model(LinearSVC(), X_train_tfidf_ngram, y_train, X_test_tfidf_ngram)
print('SVM, N-Gram Vectors   : %.4f\n' % accuracy3)

SVM, N-Gram Vectors   : 0.8210

CPU times: user 53.9 ms, sys: 30 µs, total: 53.9 ms
Wall time: 57.6 ms


In [38]:
%%time
# Support Vector Machine on Character Level TF IDF Vectors
accuracy4 = train_model(LinearSVC(), X_train_tfidf_ngram_chars, y_train, X_test_tfidf_ngram_chars)
print('SVM, CharLevel Vectors: %.4f\n' % accuracy4)

SVM, CharLevel Vectors: 0.8570

CPU times: user 497 ms, sys: 29 ms, total: 526 ms
Wall time: 528 ms


In [39]:
results.loc['Support Vector Machine'] = {
    'Count Vectors': accuracy1,
    'WordLevel TF-IDF': accuracy2,
    'N-Gram Vectors': accuracy3,
    'CharLevel Vectors': accuracy4}

### Bagging Models

In [40]:
%%time
# Bagging (Random Forest) on Count Vectors
accuracy1 = train_model(RandomForestClassifier(n_estimators = 100), X_train_count, y_train, X_test_count)
print('RF, Count Vectors    : %.4f\n' % accuracy1)

RF, Count Vectors    : 0.8270

CPU times: user 14.8 s, sys: 8.81 ms, total: 14.8 s
Wall time: 14.9 s


In [41]:
%%time
# Bagging (Random Forest) on Word Level TF IDF Vectors
accuracy2 = train_model(RandomForestClassifier(n_estimators = 100), X_train_tfidf, y_train, X_test_tfidf)
print('RF, WordLevel TF-IDF : %.4f\n' % accuracy2)

RF, WordLevel TF-IDF : 0.8355

CPU times: user 7.68 s, sys: 2.91 ms, total: 7.69 s
Wall time: 7.71 s


In [42]:
%%time
# Bagging (Random Forest) on Ngram Level TF IDF Vectors
accuracy3 = train_model(RandomForestClassifier(n_estimators = 100), X_train_tfidf_ngram, y_train, X_test_tfidf_ngram)
print('RF, N-Gram Vectors   : %.4f\n' % accuracy3)

RF, N-Gram Vectors   : 0.7835

CPU times: user 7.44 s, sys: 1.99 ms, total: 7.45 s
Wall time: 7.46 s


In [43]:
%%time
# Bagging (Random Forest) on Character Level TF IDF Vectors
accuracy4 = train_model(RandomForestClassifier(n_estimators = 100), X_train_tfidf_ngram_chars, y_train, X_test_tfidf_ngram_chars)
print('RF, CharLevel Vectors: %.4f\n' % accuracy4)

RF, CharLevel Vectors: 0.7785

CPU times: user 26.2 s, sys: 18.7 ms, total: 26.2 s
Wall time: 26.3 s


In [44]:
results.loc['Random Forest'] = {
    'Count Vectors': accuracy1,
    'WordLevel TF-IDF': accuracy2,
    'N-Gram Vectors': accuracy3,
    'CharLevel Vectors': accuracy4}

### Boosting Models

In [45]:
%%time
# Gradient Boosting on Count Vectors
accuracy1 = train_model(GradientBoostingClassifier(), X_train_count, y_train, X_test_count)
print('GB, Count Vectors    : %.4f\n' % accuracy1)

GB, Count Vectors    : 0.7990

CPU times: user 20.5 s, sys: 19.8 ms, total: 20.6 s
Wall time: 20.6 s


In [46]:
%%time
# Gradient Boosting on Word Level TF IDF Vectors
accuracy2 = train_model(GradientBoostingClassifier(), X_train_tfidf, y_train, X_test_tfidf)
print('GB, WordLevel TF-IDF : %.4f\n' % accuracy2)

GB, WordLevel TF-IDF : 0.7960

CPU times: user 14.5 s, sys: 10.9 ms, total: 14.5 s
Wall time: 14.5 s


In [47]:
%%time
# Gradient Boosting on Ngram Level TF IDF Vectors
accuracy3 = train_model(GradientBoostingClassifier(), X_train_tfidf_ngram, y_train, X_test_tfidf_ngram)
print('GB, N-Gram Vectors   : %.4f\n' % accuracy3)

GB, N-Gram Vectors   : 0.7340

CPU times: user 8.36 s, sys: 7.98 ms, total: 8.36 s
Wall time: 8.37 s


In [48]:
%%time
# Gradient Boosting on Character Level TF IDF Vectors
accuracy4 = train_model(GradientBoostingClassifier(), X_train_tfidf_ngram_chars, y_train, X_test_tfidf_ngram_chars)
print('GB, CharLevel Vectors: %.4f\n' % accuracy4)

GB, CharLevel Vectors: 0.8020

CPU times: user 2min 15s, sys: 34.1 ms, total: 2min 15s
Wall time: 2min 15s


In [49]:
results.loc['Gradient Boosting'] = {
    'Count Vectors': accuracy1,
    'WordLevel TF-IDF': accuracy2,
    'N-Gram Vectors': accuracy3,
    'CharLevel Vectors': accuracy4}

In [50]:
## What is the point of the feature engineering if we're not using it for the model
results

,Count Vectors,WordLevel TF-IDF,N-Gram Vectors,CharLevel Vectors
Naïve Bayes,0.8540,0.8600,0.8400,0.8180
Logistic Regression,0.8520,0.8730,0.8360,0.8485
Support Vector Machine,0.8345,0.8610,0.8210,0.8570
Random Forest,0.8270,0.8355,0.7835,0.7785
Gradient Boosting,0.7990,0.7960,0.7340,0.8020




---



---



> > > > > > > > > © 2021 Institute of Data


---



---



